# Machine Learning With Enron Corpus

## Goal: Use Machine Learning to Assess Fraud from Financial and Email Data 

Assessing fraud cases for any company is a tedious task that requires analyses across a vast amount of data. Fortunately most companies already have access to the data they need to combat fraud: emails and financials. However, these datasets are very large; it is extremely difficult to cipher through all of the data by hand. Having a machine assist and find a potential, fraudulent person of interest (POI) would save both time and money.

The email data set tested is the Enron Corpus, which can be found at (https://www.cs.cmu.edu/~./enron/). Enron was one of the top energy companies in the US in the early 2000s. The company eventually filed Bankruptcy largely due to fraudulent cases of insider trading and accounting scandals. It is one of the most noteworthy cases of fraud in the 20th century. Because of the scale of the company and the size of the email database, the Enron Corpus is a prime dataset for finding clues of fraud via financials and email.

In [31]:
# !/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from pprint import pprint
import numpy as np

## Explore Data and Find Important Features

In [32]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

all_features = ['poi', 'salary', 'deferral_payments', 'total_payments','loan_advances',
                'bonus', 'restricted_stock_deferred', 'deferred_income','total_stock_value',
                'expenses', 'exercised_stock_options', 'other','long_term_incentive',
                'restricted_stock', 'director_fees', 'to_messages','from_poi_to_this_person',
                'from_messages', 'from_this_person_to_poi','shared_receipt_with_poi',
                'email_poi_score']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [33]:
# Find number of data points and total POI
poi_list = list()

for name in data_dict:
    if data_dict[name]['poi']:
        poi_list.append(name)

print "Total Data Points:", len(data_dict)
print "Total POIs:", len(poi_list)

Total Data Points: 146
Total POIs: 18


In [34]:
import pandas as pd
data_dict = pickle.load(open("final_project_dataset.pkl", "r"))
df = pd.DataFrame.from_dict(data_dict, orient='index', dtype = np.float)
percent_nan_list = df.isnull().sum() / (df.isnull().sum() + df.notnull().sum())
print "\nPercent NaN Values:\n", percent_nan_list.sort_values(ascending = False)


Percent NaN Values:
loan_advances                0.972603
director_fees                0.883562
restricted_stock_deferred    0.876712
deferral_payments            0.732877
deferred_income              0.664384
long_term_incentive          0.547945
bonus                        0.438356
from_poi_to_this_person      0.410959
shared_receipt_with_poi      0.410959
to_messages                  0.410959
from_this_person_to_poi      0.410959
from_messages                0.410959
other                        0.363014
expenses                     0.349315
salary                       0.349315
exercised_stock_options      0.301370
restricted_stock             0.246575
total_payments               0.143836
total_stock_value            0.136986
email_address                0.000000
poi                          0.000000
dtype: float64


Initial exploration of the dataset revealed 146 data points with 18 POIs. Every data point consisted of features as mentioned above such as: salary, total_payment, to_messages, and expenses. 

Most features in the Enron Corpus contain NaN values, and these NaN values make up greater than 40% of each feature. NaN values represent a lack of information and weaken the overall influence and accuracy of a feature when testing for fraud in the database. There are multiple methods to handle NaN values; in this project NaN values were changed to be either the mean or median by use of a GridCV object and the Imputer function. 

In [76]:
features_list = ['poi','total_payments', 'total_stock_value','exercised_stock_options','restricted_stock']

Digging into the dataset revealed not all entries were people's names: 'SKILLING JEFFREY K' or 'LAY KENNETH L'; however both 'TOTAL' and 'THE TRAVEL AGENCY IN THE PARK' were listed as if they were actual people that worked for Enron. Because both were not living beings, I decided to remove them from calculations. 

In [46]:
### Task 2: Remove outliers
del data_dict['TOTAL']
del data_dict['THE TRAVEL AGENCY IN THE PARK']

KeyError: 'TOTAL'

After removing the outliers, three additional features were calculated but were later found to be of minimal importance: percent_to_poi, percent_from_poi, and email_poi_score. 

The thought was to assign a score to each person signifying the capacity they were in contact with a POI. The score was calculated by summing a person's percent of emails _to_ or _from_ a POI. Again, this yielded no significant gain and was discarded as a feature.

In [47]:
### Task 3: Create new feature(s)

def normalize_feature(feature, data_dict):
    # initialize high and low value for normalization function
    value_high = None
    value_low = None

    # loop through persons to find high and low values for features
    for person in data_dict:
        value = data_dict[person][feature]
        if value != 'NaN':
            # If first value in feature then assign value to variables
            if value_low == None:
                value_high = value
                value_low = value
            # look to see if value is higher or lower
            if value > value_high:
                value_high = value
            elif value < value_low:
                value_low = value

    # loop to assign normalization value
    for person in data_dict:
        value = float(data_dict[person][feature])
        # if value exists between high and low
        if (value_high >= value) and (value_low <= value):
            # if denominator isn't zero
            if value_high != value_low:
                value_norm = (value - value_low) / (value_high - value_low)
                data_dict[person][feature] = value_norm
            
# find percent emails sent to poi and percent from poi to this person
for person in data_dict:
    from_messages = data_dict[person]['from_messages']
    to_messages = data_dict[person]['to_messages']
    from_poi = data_dict[person]['from_poi_to_this_person']
    to_poi = data_dict[person]['from_this_person_to_poi']
    
    # Initialize all email_poi_score as 'NaN'
    data_dict[person]['email_poi_score'] = 'NaN'

    percent_to = float(to_poi) / float(from_messages)
    percent_from = float(from_poi) / float(to_messages)

    data_dict[person]['percent_to_poi'] = percent_to
    data_dict[person]['percent_from_poi'] = percent_from

# normailize percent_to_poi and percent_from_poi and add together
normalize_feature('percent_to_poi', data_dict)
normalize_feature('percent_from_poi', data_dict)

# add normalized percent_to_poi and percent_from_poi to create email_poi_score
for person in data_dict:
    percent_to_norm = data_dict[person]['percent_to_poi']
    percent_from_norm = data_dict[person]['percent_from_poi']

    email_poi_score = percent_to_norm + percent_from_norm
    if email_poi_score >= 0:
        data_dict[person]['email_poi_score'] = email_poi_score
        
# normalize 'email_poi_score'
normalize_feature('email_poi_score', data_dict)

In [48]:
### Store to my_dataset for easy export below.
my_dataset = data_dict

##### Hand-picked Feature Selection

Feature selection was initially hand-picked from visual aide via https://public.tableau.com/profile/diego2420#!/vizhome/Udacity/UdacityDashboard. Features were chosen based on visual clumping of POIs and non-POIs. 

Hand-picked features for determining POI:
* exercised_stock_options (high values ~ POI)
* deferred_income (low values ~ POI)
* expenses (low values ~ not POI)

In [49]:
features_handpicked = ['poi', 'exercised_stock_options', 'deferred_income', 'expenses']

##### SelectKBest Feature Selection

Features were also chosen using SelectKBest. Because top features can change from the randomness of training and testing the data, a tally was taken to determine which features appear in the top 3 features the most over 1000 trials.

SelectKBest features for determining POI:
* exercised_stock_option
* total_stock_value
* bonus


In [50]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [81]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import cross_validation
from operator import add
from heapq import nlargest

# Run loop to find how many times a feature occurs in the top 3
best_features = [0] * (len(features_list)-1)
for i in range(1000):
    # Create features and training labels
    labels, features = targetFeatureSplit(data)
    features_train, features_test, labels_train, labels_test = \
        cross_validation.train_test_split(features, labels, test_size=0.33)

    # Generate SelectKBest with k=3 features
    selector = SelectKBest(f_classif, k=1)
    selector.fit(features_train, labels_train)
    
    # Increase score of feature if it appears in the top 3   
    best_features = selector.get_support().astype(int) + best_features

    
print "In top 3:\n", best_features
# Print the top 3 features scored by which features appeared most in top 3
features_kbest = ['poi']
for e in nlargest(2, best_features):
    for index in range(len(best_features)):
        if e == best_features[index]:
            top_feature = features_list[index+1]
            if top_feature not in features_kbest:
                features_kbest.append(top_feature)

print "\nTop 3 features:\n", features_kbest        

In top 3:
[ 29 333 599  39]

Top 3 features:
['poi', 'exercised_stock_options', 'total_stock_value']


## Classifiers

To test for optimal training from the data multiple classifiers are used:
* Decision Tree
* Random Forest
* Extra Trees
* SVMs
* GaussianNB

Classifiers were tested for high scores in precision, recall, and f1. Precision is the percent of people the classifier accurately labeled as a POI. Recall is the percent of actual POIs that were correctly classified. The F1 score relates both precion and recall. F1 scores are calculated by this equation:

$$ F1 = 2 * (precision * recall)  /  (precision + recall) $$

Only classifiers with precsion and recall scores greater than or equal to .33 will be considered for the best overall classifier. Any of those best classifiers will then be ranked by highest f1 score.

During initial testings I noticed a very high variation of all scores; classifiers that yielded high precision and recall scores may have low scores on the next test. To counteract the high deviation of values, I wrote a test function to generate 1000 classifiers of each type and compare their average scores.


In [52]:
"""### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from pprint import pprint



# Create test to find average precision and recall scores
def test_prec_recall(name, clf_choice, features_list):
    precision_list = list()
    recall_list = list()
    for i in range(1000):
        ### Extract features and labels from dataset for local testing
        data = featureFormat(data_dict, features_list, sort_keys = True)
        # Create labels and features
        labels, features = targetFeatureSplit(data)

        # transform into np.array for StratifiedShuffleSplit
        features = np.array(features)
        labels = np.array(labels)

        # Shuffle and split data into training/testing sets
        sss = StratifiedShuffleSplit()
        for train_index, test_index in sss.split(features, labels):
            features_train, features_test = features[train_index], features[test_index]
            labels_train, labels_test = labels[train_index], labels[test_index]

        # Create, fit, and predict classifier
        clf = clf_choice
        clf.fit(features_train, labels_train)
        labels_pred = clf.predict(features_test)

        try:
            precision = precision_score(labels_test, labels_pred)
            recall = recall_score(labels_test, labels_pred)
            precision_list.append(precision)
            recall_list.append(recall)
        except:
            pass
    
    # F score is calculated via the mean precision and recall scores
    p_score = np.mean(precision_list)
    r_score = np.mean(recall_list)
    f_score = 2 * (p_score * r_score) / (p_score + r_score)
    
    print "\n" + "#" * 60
    print " " * 20 + name + "\n"
    print "Precision Mean Score: ", p_score
    print "Recall Mean Score: ", r_score
    print "F Score: ", f_score
    print "\n" + "#" * 60    """

'### Task 4: Try a varity of classifiers\n### Please name your classifier clf for easy export below.\n### Note that if you want to do PCA or other multi-stage operations,\n### you\'ll need to use Pipelines. For more info:\n### http://scikit-learn.org/stable/modules/pipeline.html\n\nfrom time import time\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import classification_report\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.metrics import precision_score\nfrom sklearn.metrics import recall_score\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.model_selection import StratifiedShuffleSplit\nfrom pprint import pprint\n\n\n\n# Create test to find average precision and recall scores\ndef test_prec_recall(name, clf_choice, features_list):\n    precision_list = list()\n    recall_list = list()\n    for i in range(1000):\n        ### Extract features and labels from dataset for local testing\n        data = featureFormat(data_dict, features_l

## Classifiers (Hand-picked Features)

##### Classifiers: Decision Tree (Hand-picked)

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit

pipeline_dt = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('skb', SelectKBest(f_classif)),
    ('clf', DecisionTreeClassifier()),
])

param_grid_dt = [
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [1],
        'skb__k': [1],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [2],
        'skb__k': [1,2],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [3],
        'skb__k': [1,2,3],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [4],
        'skb__k': [1,2,3,4],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    }
]        

cross_validator = StratifiedShuffleSplit()


from sklearn.model_selection import GridSearchCV
gridCV_object_dt = GridSearchCV(estimator = pipeline_dt,
                                param_grid = param_grid_dt,
                                scoring = 'f1',
                                cv=cross_validator)

# fit the data
gridCV_object_dt.fit(features, labels)

# get the best estimator
pipeline_clf_dt = gridCV_object_dt.best_estimator_

# test results
from tester import test_classifier
test_classifier(pipeline_clf_dt, my_dataset, all_features)

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('skb', SelectKBest(k=3, score_func=<function f_classif at 0x7f36a0185b18>)), (...it=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])
	Accuracy: 0.80180	Precision: 0.28615	Recall: 0.32550	F1: 0.30456	F2: 0.31679
	Total predictions: 15000	True positives:  651	False positives: 1624	False negatives: 1349	True negatives: 11376



In [78]:
print pipeline_clf_dt.named_steps['pca'].n_components_

4


##### Classifiers: Random Forest (Hand-picked)

In [79]:
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('skb', SelectKBest(f_classif)),
    ('clf', RandomForestClassifier()),
])

param_grid_rf = [
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [1],
        'skb__k': [1],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [2],
        'skb__k': [1,2],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [3],
        'skb__k': [1,2,3],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [4],
        'skb__k': [1,2,3,4],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    }
]        

cross_validator = StratifiedShuffleSplit(random_state = 0)


from sklearn.model_selection import GridSearchCV
gridCV_object_rf = GridSearchCV(estimator = pipeline_rf,
                                param_grid = param_grid_rf,
                                scoring = 'f1',
                                cv=cross_validator)

# fit the data
gridCV_object_rf.fit(features, labels)

# get the best estimator
pipeline_clf_rf = gridCV_object_rf.best_estimator_

# test results
from tester import test_classifier
test_classifier(pipeline_clf_rf, my_dataset, all_features)

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('skb', SelectKBest(k=3, score_func=<function f_classif at 0x7f36a0185b18>)), (...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
	Accuracy: 0.84973	Precision: 0.33924	Recall: 0.13400	F1: 0.19211	F2: 0.15245
	Total predictions: 15000	True positives:  268	False positives:  522	False negatives: 1732	True negatives: 12478



In [80]:
print pipeline_clf_rf.named_steps['pca'].n_components_

3


##### Classifiers: Extra Trees (Hand-picked)

In [60]:
from sklearn.ensemble import ExtraTreesClassifier

pipeline_et = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('skb', SelectKBest(f_classif)),
    ('clf', ExtraTreesClassifier()),
])

param_grid_et = [
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [1],
        'skb__k': [1],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [2],
        'skb__k': [1,2],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [3],
        'skb__k': [1,2,3],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [4],
        'skb__k': [1,2,3,4],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    }
]        

cross_validator = StratifiedShuffleSplit(random_state = 0)


from sklearn.model_selection import GridSearchCV
gridCV_object_et = GridSearchCV(estimator = pipeline_et,
                                param_grid = param_grid_et,
                                scoring = 'f1',
                                cv=cross_validator)

# fit the data
gridCV_object_et.fit(features, labels)

# get the best estimator
pipeline_clf_et = gridCV_object_et.best_estimator_

# test results
from tester import test_classifier
test_classifier(pipeline_clf_et, my_dataset, all_features)

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('skb', SelectKBest(k=2, score_func=<function f_classif at 0x7f36a0185b18>)),...timators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])
	Accuracy: 0.87167	Precision: 0.69841	Recall: 0.06600	F1: 0.12060	F2: 0.08060
	Total predictions: 15000	True positives:  132	False positives:   57	False negatives: 1868	True negatives: 12943



In [61]:
print pipeline_clf_et.named_steps['pca'].n_components_

4


##### Classifiers: SVMs (Hand-picked)

In [66]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

pipeline_svc = Pipeline([
    ('imp', Imputer()),
    ('minmaxscaler', MinMaxScaler()),
    ('pca', PCA()),
    ('skb', SelectKBest(f_classif)),
    ('clf', SVC()),
])

param_grid_svc = [
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [3],
        'skb__k': [1,2,3],
        'clf__C': [10,50,100],
        'clf__kernel': ['rbf', 'poly'],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [4],
        'skb__k': [1,2,3,4],
        'clf__C': [10,50,100],
        'clf__kernel': ['rbf', 'poly'],
    }
]

cross_validator = StratifiedShuffleSplit()

gridCV_object_svc = GridSearchCV(estimator = pipeline_svc,
                                 param_grid = param_grid_svc,
                                 scoring = 'f1',
                                 cv = cross_validator)

# fit the data
gridCV_object_svc.fit(features_train, labels_train)


# get the best estimator
pipeline_clf_svc = gridCV_object_svc.best_estimator_

# test results
from tester import test_classifier
test_classifier(pipeline_clf_svc, my_dataset, all_features)

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('skb', Sele...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
	Accuracy: 0.86000	Precision: 0.00980	Recall: 0.00050	F1: 0.00095	F2: 0.00062
	Total predictions: 15000	True positives:    1	False positives:  101	False negatives: 1999	True negatives: 12899



In [67]:
print pipeline_clf_svc.named_steps['pca'].n_components_

3


##### Classifiers: Naive Bayes (Hand-picked)

In [68]:
from sklearn.naive_bayes import GaussianNB

pipeline_nb = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('skb', SelectKBest(f_classif)),
    ('clf', RandomForestClassifier()),
])

param_grid_nb = [
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [3],
        'skb__k': [1,2,3],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    },
    {
        'imp__strategy': ['median', 'mean'],
        'pca__n_components': [4],
        'skb__k': [1,2,3,4],
        'clf__min_samples_split': [2,3,5],
        'clf__max_depth': [None,2,3],
    }
]        

cross_validator = StratifiedShuffleSplit(random_state = 0)


from sklearn.model_selection import GridSearchCV
gridCV_object_nb = GridSearchCV(estimator = pipeline_nb,
                                param_grid = param_grid_nb,
                                scoring = 'f1',
                                cv=cross_validator)

# fit the data
gridCV_object_nb.fit(features, labels)

# get the best estimator
pipeline_clf_nb = gridCV_object_nb.best_estimator_

# test results
from tester import test_classifier
test_classifier(pipeline_clf_nb, my_dataset, all_features)

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('skb', SelectKBest(k=3, score_func=<function f_classif at 0x7f36a0185b18>)), (...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
	Accuracy: 0.84900	Precision: 0.35096	Recall: 0.15600	F1: 0.21599	F2: 0.17550
	Total predictions: 15000	True positives:  312	False positives:  577	False negatives: 1688	True negatives: 12423



In [69]:
print pipeline_clf_nb.named_steps['pca'].n_components_

3


### Classifiers: Hand-picked Summary

Both the Decison Tree and Extra Trees classifiers are close to meeting the criteria of having precision and recall scores greater than .33. However, the Extra Trees Classifier has a higher precision and higher f score compared to Decision Tree.

The **Extra Trees classifier** is the best of the **hand-picked** feature testing with ** precision = .398**, ** recall = .312**, and **F score = .350**.

## Classifiers (SelectKBest Features)

##### Classifiers: Decision Tree (SelectKBest)

In [ ]:
pipeline_dtsk = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('clf', DecisionTreeClassifier(random_state = 49)),
])

param_grid_dtsk = {'pca__n_components': [2,3],
                 'imp__strategy': ['median', 'mean'],
                 'clf__min_samples_split': [2,3,5],
                 'clf__max_depth': [None,2,3],
                 }        

cross_validator = StratifiedShuffleSplit(random_state = 0)

gridCV_object_dtsk = GridSearchCV(estimator = pipeline_dtsk,
                                param_grid = param_grid_dtsk,
                                scoring = 'f1',
                                cv=cross_validator)

gridCV_object_dtsk = gridCV_object_dtsk.fit(features_train, labels_train)

In [ ]:
test_prec_recall("Decision Tree: SelectKBest", gridCV_object_dtsk.best_estimator_, features_kbest)

##### Classifiers: Random Forest (SelectKBest)

In [ ]:
pipeline_rfsk = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('clf', RandomForestClassifier(random_state = 49)),
])

param_grid_rfsk = {'pca__n_components': [2,3],
                 'imp__strategy': ['median', 'mean'],
                 'clf__n_estimators': [5,10,20],
                 'clf__min_samples_split': [2,3,5],
                 'clf__max_depth': [None,2,3],
                 }        

cross_validator = StratifiedShuffleSplit(random_state = 0)

gridCV_object_rfsk = GridSearchCV(estimator = pipeline_rfsk,
                                param_grid = param_grid_rfsk,
                                scoring = 'f1',
                                cv=cross_validator)

gridCV_object_rfsk = gridCV_object_rfsk.fit(features_train, labels_train)

In [ ]:
test_prec_recall("Random Forest: SelectKBest", gridCV_object_rfsk.best_estimator_, features_kbest)

##### Classifiers: Extra Trees (SelectKBest)

In [ ]:
pipeline_etsk = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    
    ('clf', ExtraTreesClassifier(random_state = 49)),
])

param_grid_etsk = {'pca__n_components': [2,3],
                 'imp__strategy': ['median', 'mean'],
                 'clf__n_estimators': [5,10,20],
                 'clf__min_samples_split': [2,3,5],
                 'clf__max_depth': [None,2,3],
                 }        

cross_validator = StratifiedShuffleSplit(random_state = 0)

gridCV_object_etsk = GridSearchCV(estimator = pipeline_etsk,
                                param_grid = param_grid_etsk,
                                scoring = 'f1',
                                cv=cross_validator)

gridCV_object_etsk = gridCV_object_etsk.fit(features_train, labels_train)

In [ ]:
test_prec_recall("Extra Tree: SelectKBest", gridCV_object_etsk.best_estimator_, features_kbest)

##### Classifiers: SVMs (SelectKBest)

In [ ]:
pipeline_svcsk = Pipeline([
    ('imp', Imputer()),
    ('minmaxscaler', MinMaxScaler()),
    ('pca', PCA()),
    ('clf', SVC()),
])

param_grid_svcsk = {'imp__strategy': ['median', 'mean'],
                    'clf__C': [10,50,100],
                    'clf__kernel': ['rbf', 'poly'],
                    'pca__n_components': [2,3],
                   }

cross_validator = StratifiedShuffleSplit(random_state = 0)

gridCV_object_svcsk = GridSearchCV(estimator = pipeline_svcsk,
                                 param_grid = param_grid_svcsk,
                                 scoring = 'f1',
                                 cv = cross_validator)

gridCV_object_svcsk = gridCV_object_svcsk.fit(features_train, labels_train)

In [ ]:
test_prec_recall("SVM: SelectKBest", gridCV_object_svcsk.best_estimator_, features_kbest)

##### Classifiers: Naive Bayes (SelectKBest)

In [ ]:
pipeline_nbsk = Pipeline([
    ('imp', Imputer()),
    ('pca', PCA()),
    ('clf', GaussianNB())
])

param_grid_nbsk = {'pca__n_components': [1,2,3],
                 'imp__strategy': ['median', 'mean'],
                 }

cross_validator = StratifiedShuffleSplit(random_state = 0)

gridCV_object_nbsk = GridSearchCV(estimator = pipeline_nbsk,
                                 param_grid = param_grid_nbsk,
                                 scoring = 'f1',
                                 cv = cross_validator)

gridCV_object_svcsk = gridCV_object_nbsk.fit(features_train, labels_train)

In [ ]:
test_prec_recall("Naive Bayes: SelectKBest", gridCV_object_svcsk.best_estimator_, features_kbest)

### Classifiers: SelectKBest Summary

The Decision Tree classifier completely exceeds all expections and is the clear winner of the selectkbest feature selection. The **Decision Tree** classifier with **selectkbest** attained scores of **precision = .418**, **recall = .416** and **F score = .4023**.

#### Classifiers: Hand-picked vs. SelectKBest Summary

The classifier chosen as best across both hand-picked and selectkbest is the Decision Tree classifier with the feature selection from selectkbest. Again, the selectkbest features are:
* exercised_stock_option
* total_stock_value
* bonus


In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info:
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.33)

### Validation and Parameter Tuning

##### Validation
Data validation is the series of steps it takes to make sure the data is clean and useful. Dirty data (being mislabeled or unaccurate) leads to inconclusive results no matter the outcome. 

Validation in machine learning is usually overcome by splitting the data into training and testing sets. A training size too large can overfit the classifier causing low testing results, whereas a trianing size too low can underfit the classfier, again, causing low testing results.  

The Enron Corpus was split into both training (33%) and testing (67%) via function train_test_split with 'test_size = .33'. This essentially trains the data multiple times over different partitions of the dataset using the leftover data for testing and scoring.

##### Parameter Tuning
As discussed earlier, parameters of each classifier were tuned using both a pipeline and creating a gridCV object. The gridCV object tests multiple lists of parameters and returns the parameters that maximize a scoring function. In this case all parameters were tuned to maximize F1 scores.

### Summary

The Enron Corpus is one of the largest datasets on fraud. Although the dataset isn't vast, a Decision Tree classifier appears to be a strong option in predicting fraud. 

The most important features to analyze for attempting fraud are most likely:
* exercised stock options
* total stock
* bonus

The afformetioned features combined with a Decision Tree classifier yield precision, recall, and f1 scores close to .4. Additional/different features for higher scores are desired; the current scores appear mediocre but the classifier has potential to be a starting point for detecting fraud.  

#### References

Feature Visualization:
https://public.tableau.com/profile/diego2420#!/vizhome/Udacity/UdacityDashboard

Forum Postings:
https://discussions.udacity.com/t/getting-started-with-final-project/170846

When to chose which machine learning classifier:
http://stackoverflow.com/questions/2595176/when-to-choose-which-machine-learning-classifier

GridCV and Pipeline testing:
https://discussions.udacity.com/t/webcast-builidng-models-with-gridsearchcv-and-pipelines-thursday-11-feb-2015-at-6pm-pacific-time/47412

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(gridCV_object_dtsk.best_estimator_, my_dataset, features_kbest)